# DeFi Transactions

In this notebook, we will compile all of the transaction data in to one file.

In [14]:
import requests
import json
import pandas as pd
import seaborn as sns
import cryptocompare
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from datetime import datetime
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import missingno as msno
from IPython.display import HTML
import matplotlib.dates as md
import time
import math

### Load Transaction Data

We start by loading the csv files for each transaction type.

In [15]:
df_borrows=pd.read_csv('borrows.csv')
df_borrows.head()

,amount,borrowRate,borrowRateMode,onBehalfOf,pool,reserve,timestamp,user,type,reservePriceETH,reservePriceUSD,amountUSD
0,4.150163e+04,6.274937,Variable,0x94ee9c600870c4199a1af8496eeb3087f2d1c32f,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,DAI,1621340435,0x94ee9c600870c4199a1af8496eeb3087f2d1c32f,borrow,2.852900e+14,0.994804,4.128600e+04
1,7.000000e+06,2.589628,Variable,0x51346d389ec7adf11c5b96695ef003dbb849af9a,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDT,1622477822,0x51346d389ec7adf11c5b96695ef003dbb849af9a,borrow,3.812835e+14,1.000000,7.000000e+06
2,1.500000e+04,8.802541,Variable,0x416d7f3823bc7d8dbab00056ede0325e52f0a5c2,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDC,1619775984,0x416d7f3823bc7d8dbab00056ede0325e52f0a5c2,borrow,3.611000e+14,1.004339,1.506508e+04
3,8.193190e+03,48.747052,Stable,0x78cbc5e9eb4f96e5ced75819e76634a768b57e07,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDC,1615481632,0x78cbc5e9eb4f96e5ced75819e76634a768b57e07,borrow,5.562201e+14,0.999391,8.188200e+03
4,1.100000e+04,3.225055,Variable,0xbed4dbd30fd3aed29c2d133fddb611f8aa517c6b,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDT,1626914745,0xbed4dbd30fd3aed29c2d133fddb611f8aa517c6b,borrow,4.971100e+14,1.000000,1.100000e+04


In [16]:
df_repays=pd.read_csv('repays.csv')
df_repays.head()

,amount,onBehalfOf,pool,reserve,timestamp,user,type,reservePriceETH,reservePriceUSD,amountUSD
0,1.801803e+06,0x0bbe643d5d9dd0498d0c9546f728504a4aab78f4,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDC,1616417822,0x0bbe643d5d9dd0498d0c9546f728504a4aab78f4,repay,5.566700e+14,0.995654,1.793972e+06
1,7.196676e+02,0x02e76c933b98c46fabd87fb30c14c7a623319bbe,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDT,1611491659,0x02e76c933b98c46fabd87fb30c14c7a623319bbe,repay,7.542830e+14,1.000000,7.196676e+02
2,7.040368e+03,0xf2b7f6f9e11f65d7a244363dfd32cdb66100fd6a,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,SUSD,1617162703,0xf2b7f6f9e11f65d7a244363dfd32cdb66100fd6a,repay,5.488300e+14,1.005494,7.079051e+03
3,7.400043e+03,0xc67d4c582cbc48eb3701b9ab9a0ccdbc23e5aebc,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,DAI,1609463063,0xc67d4c582cbc48eb3701b9ab9a0ccdbc23e5aebc,repay,1.360102e+15,1.003491,7.425878e+03
4,1.210160e+05,0x208b82b04449cd51803fae4b1561450ba13d9510,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDC,1621559868,0x208b82b04449cd51803fae4b1561450ba13d9510,repay,3.446828e+14,1.001534,1.212016e+05


In [17]:
df_deposits=pd.read_csv('deposits.csv')
df_deposits.head()

,amount,onBehalfOf,pool,reserve,timestamp,user,type,reservePriceETH,reservePriceUSD,amountUSD
0,0.005386,0xf26c81b576eaeeb532634a0ec4256e3bf64db139,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,AAVE,1626855185,0x2801a71605b5e25816235c7f3cb779f4c9dd60ee,deposit,1.290933e+17,2.421049e+02,1.304095e+00
1,0.500000,0xd1da235c93496317a9dd2f5d22b9fd622a0af1c4,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,WETH,1627601050,0xcc9a0b7c43dc2a5f023bb9b738e45b0ef6b06e04,deposit,1.000000e+00,2.390167e-15,1.195084e-15
2,460.000000,0x346f1d297c98c28574742b067b67a80cda2dc0d9,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,WETH,1628959994,0xcc9a0b7c43dc2a5f023bb9b738e45b0ef6b06e04,deposit,1.000000e+00,3.270724e-15,1.504533e-12
3,7500.000000,0x762b4a03d4d8ea0f81d2a0254770c9df18361051,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,DAI,1618692791,0x762b4a03d4d8ea0f81d2a0254770c9df18361051,deposit,4.221698e+14,1.000616e+00,7.504617e+03
4,27991.243002,0x6082fb69b9886b4cc55d23f87510f7c0841dbc3f,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDT,1610390681,0x6082fb69b9886b4cc55d23f87510f7c0841dbc3f,deposit,9.898730e+14,1.000000e+00,2.799124e+04


In [18]:
df_redeems=pd.read_csv('redeems.csv')
df_redeems.head()

,amount,onBehalfOf,pool,reserve,timestamp,user,type,reservePriceETH,reservePriceUSD,amountUSD
0,2.100000e+06,0xef5c8dfb7abaa3acd759d77fb156cea315eb69b8,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDC,1619717013,0xef5c8dfb7abaa3acd759d77fb156cea315eb69b8,redeem,3.653611e+14,0.997056,2.093817e+06
1,6.703367e+01,0xefeb6afa02d5b368005176c9d8d849aa58a98be2,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,CRV,1626670785,0xefeb6afa02d5b368005176c9d8d849aa58a98be2,redeem,7.479100e+14,1.417037,9.498922e+01
2,1.999998e+04,0x63a3f444e97d14e671e7ee323c4234c8095e3516,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDC,1624224687,0x63a3f444e97d14e671e7ee323c4234c8095e3516,redeem,4.439400e+14,1.003846,2.007690e+04
3,2.002598e+04,0x04956efd10b49c1a7dad15e444092faeb521bddc,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDC,1623371599,0x04956efd10b49c1a7dad15e444092faeb521bddc,redeem,4.067300e+14,1.000057,2.002713e+04
4,9.525850e+05,0xfb9e3b8dd7201258ab17e5173efe935c0be611ec,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDC,1628694237,0xfb9e3b8dd7201258ab17e5173efe935c0be611ec,redeem,3.084196e+14,0.999415,9.520282e+05


In [19]:
df_liquidations=pd.read_csv('liquidations.csv')
df_liquidations.head()

,collateralAmount,collateralReserve,pool,principalAmount,principalReserve,timestamp,user,type,reservePriceETHPrincipal,reservePriceUSDPrincipal,reservePriceETHCollateral,reservePriceUSDCollateral,amountUSDPincipal,amountUSDCollateral
0,0.330855,WETH,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,639.170000,GUSD,1626124715,0x3424b949dbc5d2d2654cb6fc622c4d723b2e8fb4,liquidation,3.268884e+14,0.662549,1.000000e+00,2.026835e-15,423.481393,6.705890e-16
1,33825.732222,ENJ,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,55144.569141,USDT,1619145033,0x41a792401694fc00467207cfeb9b5f09aa2b834a,liquidation,4.568000e+14,1.000000,4.568000e+14,5.576372e-01,55144.569141,1.886249e+04
2,6345.434466,USDC,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,0.082080,YFI,1621319875,0xc6145175d51edfd6383851dc101f5872a79d7829,liquidation,2.068914e+19,73178.919426,2.810000e+14,9.939162e-01,6006.505124,6.306830e+03
3,84.893205,WETH,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,117034.106539,BUSD,1614324006,0x01acb3804ba9c42111c6e9c127831eb486ca1ac7,liquidation,6.908300e+14,1.012487,1.000000e+00,1.465609e-15,118495.510575,1.244203e-13
4,3749.460623,LINK,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,52067.659439,USDC,1621788289,0x01214a235c6767040c8fef6bd33519a6c7ad79ae,liquidation,5.739400e+14,1.012537,8.767150e+15,1.546688e+01,52720.422939,5.799247e+04


In [20]:
df_swaps=pd.read_csv('swaps.csv')
df_swaps.head()

,borrowRateModeFrom,borrowRateModeTo,pool,reserve,stableBorrowRate,timestamp,user,variableBorrowRate,type
0,Stable,Variable,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,DAI,8.988390,1613216311,0xd0698d231d4b65b97a3df9c16aafda8d9b0bda41,3.976780,swap
1,Stable,Variable,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDC,14.096271,1613943756,0x5c2acae2bfbf75b7a2f045dc7f4326bc4f0b048d,9.096271,swap
2,Variable,Stable,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDT,12.567462,1617270491,0x7f6fa589d48d42fd290a976abd9634c32405ec0b,4.567462,swap
3,Variable,Stable,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDC,8.996132,1614461983,0xccec96a03dda5c438996701f26bef600366a137b,3.992264,swap
4,Variable,Stable,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,DAI,11.969824,1619275604,0xf8523151211e6cfb2d285b9b92f2ee517722253b,3.939648,swap


### Compile Transaction Data

Next, we merge them all together in to one big DataFrame.

In [21]:
#merge dataframes
df = df_borrows.append(df_repays).append(df_liquidations).append(df_deposits).append(df_redeems).append(df_swaps)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 481519 entries, 0 to 999
Data columns (total 26 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   amount                     474230 non-null  float64
 1   borrowRate                 94977 non-null   float64
 2   borrowRateMode             94977 non-null   object 
 3   onBehalfOf                 474230 non-null  object 
 4   pool                       481519 non-null  object 
 5   reserve                    475230 non-null  object 
 6   timestamp                  481519 non-null  int64  
 7   user                       481519 non-null  object 
 8   type                       481519 non-null  object 
 9   reservePriceETH            474230 non-null  float64
 10  reservePriceUSD            474230 non-null  float64
 11  amountUSD                  474230 non-null  float64
 12  collateralAmount           6289 non-null    float64
 13  collateralReserve          6289 

In [22]:
df.head()

,amount,borrowRate,borrowRateMode,onBehalfOf,pool,reserve,timestamp,user,type,reservePriceETH,...,reservePriceETHPrincipal,reservePriceUSDPrincipal,reservePriceETHCollateral,reservePriceUSDCollateral,amountUSDPincipal,amountUSDCollateral,borrowRateModeFrom,borrowRateModeTo,stableBorrowRate,variableBorrowRate
0,4.150163e+04,6.274937,Variable,0x94ee9c600870c4199a1af8496eeb3087f2d1c32f,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,DAI,1621340435,0x94ee9c600870c4199a1af8496eeb3087f2d1c32f,borrow,2.852900e+14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7.000000e+06,2.589628,Variable,0x51346d389ec7adf11c5b96695ef003dbb849af9a,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDT,1622477822,0x51346d389ec7adf11c5b96695ef003dbb849af9a,borrow,3.812835e+14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.500000e+04,8.802541,Variable,0x416d7f3823bc7d8dbab00056ede0325e52f0a5c2,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDC,1619775984,0x416d7f3823bc7d8dbab00056ede0325e52f0a5c2,borrow,3.611000e+14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8.193190e+03,48.747052,Stable,0x78cbc5e9eb4f96e5ced75819e76634a768b57e07,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDC,1615481632,0x78cbc5e9eb4f96e5ced75819e76634a768b57e07,borrow,5.562201e+14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.100000e+04,3.225055,Variable,0xbed4dbd30fd3aed29c2d133fddb611f8aa517c6b,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDT,1626914745,0xbed4dbd30fd3aed29c2d133fddb611f8aa517c6b,borrow,4.971100e+14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Finally, we save this DataFrame as a csv file.

In [23]:
df.to_csv("transactions.csv",index=False)

In [24]:
df_borrows[df_borrows['reserve']=='AmmWETH']

,amount,borrowRate,borrowRateMode,onBehalfOf,pool,reserve,timestamp,user,type,reservePriceETH,reservePriceUSD,amountUSD
144,3.8500,6.814482,Variable,0x87ea2c7f53c84242bb6496923f940cc5ff64a7cf,0xacc030ef66f9dfeae9cbb0cd1b25654b82cfa8d5,AmmWETH,1622346358,0xcc9a0b7c43dc2a5f023bb9b738e45b0ef6b06e04,borrow,1.0,2.289430e-15,8.814304e-15
184,363.0000,18.235171,Variable,0x64bca4dcafb3f889f2df9f97e7a6a6b297acb8e1,0xacc030ef66f9dfeae9cbb0cd1b25654b82cfa8d5,AmmWETH,1619785506,0xcc9a0b7c43dc2a5f023bb9b738e45b0ef6b06e04,borrow,1.0,2.749519e-15,9.980753e-13
1130,85.0000,86.883031,Variable,0xd5597936dbbd1b31f70d7a2024409b7a4d032680,0xacc030ef66f9dfeae9cbb0cd1b25654b82cfa8d5,AmmWETH,1616614969,0xcc9a0b7c43dc2a5f023bb9b738e45b0ef6b06e04,borrow,1.0,1.654232e-15,1.406098e-13
1627,0.1500,0.325235,Variable,0x8f9d959adde5f1e9a7166eeec87acd43cc7529f9,0xacc030ef66f9dfeae9cbb0cd1b25654b82cfa8d5,AmmWETH,1627198679,0xcc9a0b7c43dc2a5f023bb9b738e45b0ef6b06e04,borrow,1.0,2.165287e-15,3.247931e-16
1655,47.7700,0.421084,Variable,0x9e301190d0bdf64e287d6205d86c488fbd37d4f8,0xacc030ef66f9dfeae9cbb0cd1b25654b82cfa8d5,AmmWETH,1628396180,0xcc9a0b7c43dc2a5f023bb9b738e45b0ef6b06e04,borrow,1.0,3.101735e-15,1.481699e-13
...,...,...,...,...,...,...,...,...,...,...,...,...
94460,0.0869,0.324810,Variable,0x0f38dd12eaba3e19799840263919bb21aa5433ff,0xacc030ef66f9dfeae9cbb0cd1b25654b82cfa8d5,AmmWETH,1627103959,0xcc9a0b7c43dc2a5f023bb9b738e45b0ef6b06e04,borrow,1.0,2.139754e-15,1.859446e-16
94576,2.1000,0.311009,Variable,0x3638d7bcafe5deeeb2e8f834b7425363997c81fc,0xacc030ef66f9dfeae9cbb0cd1b25654b82cfa8d5,AmmWETH,1625883791,0xcc9a0b7c43dc2a5f023bb9b738e45b0ef6b06e04,borrow,1.0,2.157963e-15,4.531722e-15
94867,1.0000,0.447953,Variable,0x7d29d1a50a460a41bfa6b8e2f09100a27b432756,0xacc030ef66f9dfeae9cbb0cd1b25654b82cfa8d5,AmmWETH,1628965564,0xcc9a0b7c43dc2a5f023bb9b738e45b0ef6b06e04,borrow,1.0,3.235333e-15,3.235333e-15
94906,50.0000,29.871344,Variable,0x7c132bde28ad1e0101d45121757d7c6c2e8a550a,0xacc030ef66f9dfeae9cbb0cd1b25654b82cfa8d5,AmmWETH,1617089625,0xcc9a0b7c43dc2a5f023bb9b738e45b0ef6b06e04,borrow,1.0,1.811496e-15,9.057479e-14


In [25]:
df_borrows['reserve'].unique()

array(['DAI', 'USDT', 'USDC', 'WETH', 'UNI', 'GUSD', 'AmmUSDT', 'BUSD',
       'YFI', 'MANA', 'AMPL', 'AmmUSDC', 'SUSD', 'WBTC', 'CRV', 'TUSD',
       'AmmDAI', 'AmmWETH', 'LINK', 'MKR', 'XSUSHI', 'BAL', 'SNX', 'RAI',
       'BAT', 'REN', 'KNC', 'AmmWBTC', 'ZRX', 'ENJ', 'PAX', 'RENFIL',
       'AAVE'], dtype=object)

In [29]:
df['user'].head()

0    0x94ee9c600870c4199a1af8496eeb3087f2d1c32f
1    0x51346d389ec7adf11c5b96695ef003dbb849af9a
2    0x416d7f3823bc7d8dbab00056ede0325e52f0a5c2
3    0x78cbc5e9eb4f96e5ced75819e76634a768b57e07
4    0xbed4dbd30fd3aed29c2d133fddb611f8aa517c6b
Name: user, dtype: object